# Capstone Project
This notebook and repo will be used for IBM's data science professional capstone course on coursera. 

In [2]:
# import dependencies
import pandas as pd 
import numpy as np

In [4]:
# test line
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
